In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input,Flatten,Dense,Dropout
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import string
import seaborn as sns
import pickle
import os


In [2]:
train_ds = image_dataset_from_directory(
    directory=r'C:\Users\DELL\Downloads\char_recog\augmented_images\augmented_images1',
    labels='inferred',
    label_mode='int',
    image_size=(256, 256),
    shuffle=True
)

test_ds1 = image_dataset_from_directory(
    directory=r'C:\Users\DELL\Downloads\char_recog\handwritten-english-characters-and-digits\combined_folder\train',
    labels='inferred',
    label_mode='int',
    image_size=(256, 256),
    shuffle=True
)

test_ds2 = image_dataset_from_directory(
    directory=r'C:\Users\DELL\Downloads\char_recog\handwritten-english-characters-and-digits\combined_folder\test',
    labels='inferred',
    label_mode='int',
    image_size=(256, 256),
    shuffle=True
)

test_ds = test_ds1.concatenate(test_ds2)


NotFoundError: Could not find directory C:\Users\DELL\Downloads\char_recog\augmented_images\augmented_images1

In [4]:
conv_base = VGG19(
    weights = 'imagenet',
    include_top = False,
    input_shape = (256, 256, 3)
)



80134624/80134624 [==============================] - 49s 1us/step


In [5]:
conv_base.trainable = False


In [6]:
conv_base.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [7]:
model = Sequential()
model.add(Input(shape=(256,256,3)))
model.add(conv_base)
model.add(Flatten())
model.add(Dense(224,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(416,activation='sigmoid'))
model.add(Dropout(0.1))
model.add(Dense(62,activation='softmax'))

In [8]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 8, 8, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 224)               7340256   
                                                                 
 dropout (Dropout)           (None, 224)               0         
                                                                 
 dense_1 (Dense)             (None, 416)               93600     
                                                                 
 dropout_1 (Dropout)         (None, 416)               0         
                                                                 
 dense_2 (Dense)             (None, 62)                2

In [9]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [11]:
history = model.fit(train_ds, epochs=15, validation_data=test_ds)


Epoch 1/15
 33/427 [=>............................] - ETA: 34:09 - loss: 3.7746 - accuracy: 0.0881

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(train_ds)
print("Loss:", loss)
print("Accuracy:",accuracy)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Loss:",loss)
print("Accuracy:",accuracy)

In [ ]:
def plot_predictions(generator, model):
    # Retrieve a batch of images and true labels from the generator
    images, true_labels = next(iter(generator))

    # Extract class names from the generator
    digits = [str(i) for i in range(10)]
    uppercase = [i for i in string.ascii_uppercase]
    lowercase = [i for i in string.ascii_lowercase]
    class_names = digits + uppercase + lowercase

    # Convert TensorFlow tensors to NumPy arrays if necessary
    if isinstance(true_labels, tensorflow.Tensor):
        true_labels = true_labels.numpy()
    if isinstance(images, tensorflow.Tensor):
        images = images.numpy()

    # Convert true_labels to class indices if they are one-hot encoded
    if true_labels.ndim > 1 and true_labels.shape[1] > 1:
        true_labels = np.argmax(true_labels, axis=1)

    # Ensure true_labels are integers
    true_labels = true_labels.astype(int)

    # Make predictions on the batch of images
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1).astype(int)

    # Check dimensions and types
    print(f'Images shape: {images.shape}')
    print(f'True labels shape: {true_labels.shape}, dtype: {true_labels.dtype}')
    print(f'Predicted classes shape: {predicted_classes.shape}, dtype: {predicted_classes.dtype}')

    # Set up the plot
    num_images = len(images)
    print(f'Number of images: {num_images}')

    # Calculate the number of rows needed
    num_cols = 6
    num_rows = int(np.ceil(num_images / num_cols))

    # Create subplots
    fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(20, 5 * num_rows))

    # Flatten axes array for easy iteration
    axes = axes.flatten()

    for i in range(num_images):
        ax = axes[i]
        img = images[i]
        true_label_index = true_labels[i]
        predicted_label_index = predicted_classes[i]

        # Rescale the image if it was scaled during preprocessing
        img = img.astype(np.uint8)

        # Plot the image
        ax.imshow(img)
        ax.axis('off')

        # Get class names
        true_label_name = class_names[true_label_index]
        predicted_label_name = class_names[predicted_label_index]

        # Display true and predicted class names as text
        ax.set_title(f'True: {true_label_name}\nPred: {predicted_label_name}')

    # Hide any unused subplots
    for j in range(num_images, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
plot_predictions(test_ds, model)


In [ ]:
plot_predictions(train_ds, model)


In [ ]:
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:
cm = confusion_matrix(y_true, y_pred)
class_names = train_ds.class_names
num_classes = len(np.unique(y_true))
cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

plt.figure(figsize=(20, 20))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=True, square=True, 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()